<a href="https://colab.research.google.com/github/rdkdaniel/The-Swahili-Project/blob/main/The_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Collection**

## **The Issue**

In [ ]:
#Scrapping works well with stuctured data, like tables etc.
#Also, there is the issue of labelling:
#Using myself or someelse may result in bias (interpretation of swahili words or sentences based on given labels)
  #Also, this will be tedious and costly (time)

In [ ]:
#Any other alternative???

# **Data Collection Methods**

## **1.0 Using Scrapy**

### **1.0.0 Libraries**

In [1]:
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

'3.7.15'

### **1.0.1 More Libraries - Scrapy**

In [2]:
#Import Scrapy
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 4.0 MB/s 
     |████████████████████████████████| 254 kB 63.0 MB/s 
     |████████████████████████████████| 4.0 MB 67.0 MB/s 
     |████████████████████████████████| 3.1 MB 42.3 MB/s 
     |████████████████████████████████| 57 kB 5.2 MB/s 
     |████████████████████████████████| 93 kB 2.8 MB/s 
     |████████████████████████████████| 74 kB 3.8 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11958 sha256=1779981940e67a4c5bd658dd23e7fdd2039e042a30460cc71bc78f171860a5c0
  Stored in directory: /root/.cache/pip/wheels/c9/d6/6a/de198d890277cde60ca3dbebe7ae592d3b381c7d9bb2455f4d
Successfully built PyDispatcher


### **1.0.2 Setting Up the Pipeline**

In [3]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

### **1.0.3 Defining the Spider**

In [4]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'https://www.advance-africa.com/KCSE-Past-Papers-2021-Kiswahili-Karatasi-ya-3.html',
        'https://www.advance-africa.com/KCSE-Past-Papers-2021-Kiswahili-Karatasi-ya-2.html',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoteresult.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

### **1.0.4 Starting the Crawler**

In [5]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

INFO:scrapy.utils.log:Scrapy 2.7.0 started (bot: scrapybot)
2022-10-31 02:15:32 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
2022-10-31 02:15:32 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
INFO:scrapy.crawler:Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2022-10-31 02:15:32 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGE

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.advance-africa.com/KCSE-Past-Papers-2021-Kiswahili-Karatasi-ya-2.html> (referer: None)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.advance-africa.com/KCSE-Past-Papers-2021-Kiswahili-Karatasi-ya-3.html> (referer: None)
INFO:scrapy.core.engine:Closing spider (finished)
INFO:scrapy.statscollectors:Dumping Scrapy stats:
{'downloader/request_bytes': 576,
 'downloader/request_count': 2,
 'downloader/request_method_count/GET': 2,
 'downloader/response_bytes': 383968,
 'downloader/response_count': 2,
 'downloader/response_status_count/200': 2,
 'elapsed_time_seconds': 1.785463,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2022, 10, 31, 2, 15, 34, 843849),
 'httpcompression/response_bytes': 2436574,
 'httpcompression/response_count': 2,
 'memusage/max': 139415552,
 'memusage/startup': 139415552,
 'response_received_count': 2,
 'scheduler/dequeued': 2,
 'scheduler/dequeued/memory': 2,
 'scheduler/enqueued': 2,

### **1.0.5 Checking the Files**

In [6]:
ll quoteresult.*

-rw-r--r-- 1 root 0 Oct 31 02:15 quoteresult.jl
-rw-r--r-- 1 root 0 Oct 31 02:15 quoteresult.json


In [7]:
!tail -n 2 quoteresult.jl

In [8]:
!tail -n 2 quoteresult.json

### **1.0.6 Turning them to Dataframes**

In [9]:
import pandas as pd


INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [13]:
data= pd.read_json('quoteresult.json', lines=True, orient='records')

In [14]:
data

""


## **2.0 Scrapy 2.0**

### **2.0.0 Libraries**

In [ ]:
!pip install scrapy
!pip install crochet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### **2.0.1 Scrapping the Webpage**

In [ ]:
# scrape webpage
import scrapy
from scrapy.crawler import CrawlerRunner
# text cleaning
import re
# Reactor restart
from crochet import setup, wait_for
setup()

Exception in thread CrochetReactor:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/crochet/_eventloop.py", line 372, in <lambda>
    target=lambda: self._reactor.run(installSignalHandlers=False),
  File "/usr/local/lib/python3.7/dist-packages/twisted/internet/base.py", line 1317, in run
    self.startRunning(installSignalHandlers=installSignalHandlers)
  File "/usr/local/lib/python3.7/dist-packages/twisted/internet/base.py", line 1299, in startRunning
    ReactorBase.startRunning(cast(ReactorBase, self))
  File "/usr/local/lib/python3.7/dist-packages/twisted/internet/base.py", line 843, in startRunning
    raise error.ReactorNotRestartable()
twisted.internet.error.ReactorNotRestartable



In [ ]:
class QuotesToCsv(scrapy.Spider):
    """scrape first line of  quotes from `wikiquote` by 
    Maynard James Keenan and save to json file"""
    name = "MJKQuotesToCsv"
    start_urls = [
        'https://en.wikiquote.org/wiki/Swahili_proverbs',
    ]
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.ExtractFirstLine': 1
        },
        'FEEDS': {
            'quotes.csv': {
                'format': 'csv',
                'overwrite': True
            }
        }
    }

    def parse(self, response):
        """parse data from urls"""
        for quote in response.css('div.mw-parser-output > ul > li'):
            yield {'quote': quote.extract()}


class ExtractFirstLine(object):
    def process_item(self, item, spider):
        """text processing"""
        lines = dict(item)["quote"].splitlines()
        first_line = self.__remove_html_tags__(lines[0])

        return {'quote': first_line}

    def __remove_html_tags__(self, text):
        """remove html tags from string"""
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)

@wait_for(10)
def run_spider():
    """run spider with MJKQuotesToCsv"""
    crawler = CrawlerRunner()
    d = crawler.crawl(QuotesToCsv)
    return d

### **2.0.2 Initializing the Crawler**

In [ ]:
process = CrawlerProcess()
process.crawl(QuotesToCsv)
process.start()

INFO:scrapy.utils.log:Scrapy 2.7.0 started (bot: scrapybot)
2022-10-27 11:14:37 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
2022-10-27 11:14:37 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
INFO:scrapy.crawler:Overridden settings:
{}
2022-10-27 11:14:37 [scrapy.crawler] INFO: Overridden settings:
{}
DEBUG:scrapy.utils.log:Using reactor: twisted.internet.epollreactor.EPollReactor
2022-10-27 11:14:37 [scrapy.

ReactorNotRestartable: ignored

## **3.0 Scrapy 3.0**

### **3.0.0 Libraries**

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess

### **3.0.1 The Spider**

In [ ]:
class MarilynMansonQuotes(scrapy.Spider):
    name = "MarilynMansonQuotes"
    start_urls = [
        'https://en.wikiquote.org/wiki/Swahili_proverbs',
    ]
    
    def parse(self, response):
        for quote in response.css('div.mw-parser-output > ul > li'):
            yield {'quote': quote.extract()}

process = CrawlerProcess()
process.crawl(MarilynMansonQuotes)
process.start()

INFO:scrapy.utils.log:Scrapy 2.7.0 started (bot: scrapybot)
2022-10-27 11:22:47 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
2022-10-27 11:22:47 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
INFO:scrapy.crawler:Overridden settings:
{}
2022-10-27 11:22:47 [scrapy.crawler] INFO: Overridden settings:
{}
DEBUG:scrapy.utils.log:Using reactor: twisted.internet.epollreactor.EPollReactor
2022-10-27 11:22:47 [scrapy.

ReactorNotRestartable: ignored

### **3.0.2 Processing pipeline**

In [16]:
import re

class ExtractFirstLine(object):
def process_item(self, item, spider):
        lines = dict(item)["quote"].splitlines()
        first_line = self.__remove_html_tags__(lines[0])
        return {'quote': first_line}
def __remove_html_tags__(self, text):
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)

IndentationError: ignored

https://www.mikulskibartosz.name/how-to-scrape-a-single-web-page-using-scrapy-in-jupyter-notebook/

## **4.0 Scrapy 4.0**

### **4.0.0 Libraries**

In [1]:
# refer to: https://docs.scrapy.org/en/latest/intro/tutorial.html
# install scrapy
!pip install Scrapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 5.3 MB/s 
     |████████████████████████████████| 57 kB 5.0 MB/s 
     |████████████████████████████████| 3.1 MB 49.2 MB/s 
     |████████████████████████████████| 4.0 MB 38.0 MB/s 
     |████████████████████████████████| 93 kB 2.5 MB/s 
     |████████████████████████████████| 254 kB 67.3 MB/s 
     |████████████████████████████████| 74 kB 3.2 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11958 sha256=fb2d56eef65753cfa83231f0e9e89ee028b6a45b95f5d38b851ac97151355ece
  Stored in directory: /root/.cache/pip/wheels/c9/d6/6a/de198d890277cde60ca3dbebe7ae592d3b381c7d9bb2455f4d
Successfully built PyDispatcher


### **4.0.1 Others**

In [18]:
# create files for learning
!scrapy startproject firstproject

New Scrapy project 'firstproject', using template directory '/usr/local/lib/python3.7/dist-packages/scrapy/templates/project', created in:
    /content/firstproject

You can start your first spider with:
    cd firstproject
    scrapy genspider example example.com


In [19]:
# get the current working directory
import os
os.getcwd()

'/content'

In [20]:
# change working directories
os.chdir('/content/firstproject/firstproject/spiders')
os.getcwd()

'/content/firstproject/firstproject/spiders'

In [21]:
# create quotes_spider.py and save it under the firstproject/firstproject/spiders directory
%%writefile -a quotes_spider.py
import scrapy

class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        urls = [
            'https://quotes.toscrape.com/page/1/',
            'https://quotes.toscrape.com/page/2/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = f'quotes-{page}.html'
        with open(filename, 'wb') as f:
            f.write(response.body)
        self.log(f'Saved file {filename}')

Writing quotes_spider.py


In [22]:
!scrapy crawl quotes

2022-10-31 02:30:40 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: firstproject)
2022-10-31 02:30:40 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
2022-10-31 02:30:40 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'firstproject',
 'NEWSPIDER_MODULE': 'firstproject.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['firstproject.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2022-10-31 02:30:40 [asyncio] DEBUG: Using selector: EpollSelector
2022-10-31 02:30:40 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2022-10-31 02:30:40 [scrapy.utils.log] DEBUG: Using asyncio event l

In [23]:
!scrapy shell 'https://quotes.toscrape.com'

2022-10-31 02:30:57 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: firstproject)
2022-10-31 02:30:57 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
2022-10-31 02:30:57 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'firstproject',
 'DUPEFILTER_CLASS': 'scrapy.dupefilters.BaseDupeFilter',
 'LOGSTATS_INTERVAL': 0,
 'NEWSPIDER_MODULE': 'firstproject.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['firstproject.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2022-10-31 02:30:57 [asyncio] DEBUG: Using selector: EpollSelector
2022-10-31 02:30:57 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.Asyncio

## **5.0 Scrapy 5.0**

### **5.1 Libraries loaded above**

### **5.2 Others**

In [2]:
# create files for learning
!scrapy startproject firstproject

New Scrapy project 'firstproject', using template directory '/usr/local/lib/python3.7/dist-packages/scrapy/templates/project', created in:
    /content/firstproject

You can start your first spider with:
    cd firstproject
    scrapy genspider example example.com


In [3]:
# get the current working directory
import os
os.getcwd()

'/content'

In [4]:
# change working directories
os.chdir('/content/firstproject/firstproject/spiders')
os.getcwd()

'/content/firstproject/firstproject/spiders'

In [5]:
# create quotes_spider.py and save it under the firstproject/firstproject/spiders directory
%%writefile -a quotes_spider.py
import scrapy

class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        urls = [
            'https://www.advance-africa.com/KCSE-Past-Papers-2021-Kiswahili-Karatasi-ya-3.html',
            'https://www.advance-africa.com/KCSE-Past-Papers-2021-Kiswahili-Karatasi-ya-2.html',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = f'quotes-{page}.html'
        with open(filename, 'wb') as f:
            f.write(response.body)
        self.log(f'Saved file {filename}')

Writing quotes_spider.py


In [6]:
!scrapy crawl quotes

2022-10-31 04:50:51 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: firstproject)
2022-10-31 04:50:51 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
2022-10-31 04:50:51 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'firstproject',
 'NEWSPIDER_MODULE': 'firstproject.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['firstproject.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2022-10-31 04:50:51 [asyncio] DEBUG: Using selector: EpollSelector
2022-10-31 04:50:51 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2022-10-31 04:50:51 [scrapy.utils.log] DEBUG: Using asyncio event l

In [ ]:
!scrapy shell 'https://www.advance-africa.com/KCSE-Past-Papers.htmlquote = response.css("div.quote")[0]'